In [6]:
import requests
import os
from dotenv import load_dotenv
load_dotenv()
from urllib.parse import urlencode
import json

Заходим в аккаунт Авито, тариф "Расширенный" или "Максимальный", переходим в раздел "Профиль и настройки" - "Интеграции и API" в параграфе "Собственная разработка" нажимаем на кнопку "Получить ключи". 
Client_id: '<CLIENT_ID>'
Client_secret: '<CLIENT_SECRET>'
Документация по API Avito: https://developers.avito.ru/api-catalog/auth/documentation#
Используем метод client_credentials (персональная авторизация)
Важно: время действия токена ограничено — 24 часа с момента его получения. 

Сохраняем все данные в .env

In [ ]:
client_secret = os.getenv('client_secret')
Client_id = os.getenv('Client_id')

Для работы с API от своего лица необходимо получить токен авторизации — отдельный для каждой учетной записи на Авито.
1. Получаем client_id и client_secret в личном кабинете (Получение описано выше)
2. Выписываем себе токен:
import requests
data = {
    'grant_type': 'client_credentials',
    'client_id': '<CLIENT_ID>',
    'client_secret': '<CLIENT_SECRET>',
}
response = requests.post('https://api.avito.ru/token/', data=data)

3. Далее выполняем запросы к API с заголовком Authorization: Bearer <ACCESS_TOKEN>
Время действия токена ограничено - 24 часа с момента его получения. После этого вам необходимо получить новый токен.

In [7]:
#2. Выписываем себе токен
data = {
    'grant_type': 'client_credentials',
    'client_id': Client_id,
    'client_secret': client_secret,
}
response = requests.post('https://api.avito.ru/token/', data=data)

In [ ]:
# Вытаскиваем токен из полученного ответа и сохраняем его в переменной 
access_token = response.json()['access_token']

In [15]:
#3. Выполняем запросы с заголовком. Все запросы делаются по общему стандарту:
# https://<адрес-сервера>/method/<имя-API-метода>?<параметры>
headers = {'Authorization': f'Bearer {access_token}'}
res = requests.get('https://api.avito.ru/core/v1/accounts/self', headers=headers)

In [ ]:
# Получение информации об авторизованном пользователе и сохраняем его айди
reg_user_id = res.json()['id']

In [ ]:
# Получение баланса кошелька пользователя
headers = {'Authorization': f'Bearer {access_token}'}
user_id = reg_user_id
res = requests.get(f'https://api.avito.ru/core/v1/accounts/{user_id}/balance/', headers=headers)
res.json()

In [ ]:
# Получение истории операций пользователя
headers = {'Authorization': f'Bearer {access_token}'}
body={
    "dateTimeFrom": "2024-12-01T00:00:00",
    "dateTimeTo": "2024-12-14T00:00:00"
}
res = requests.post(f'https://api.avito.ru/core/v1/accounts/operations_history/', headers=headers, json=body)
res.json()


In [ ]:
# Звонки по времени
headers = {'Authorization': f'Bearer {access_token}'}
body={
  "dateTimeFrom": "2024-12-12T00:00:00Z",
  "dateTimeTo": "2024-12-14T23:59:59Z",
  "limit": 100,
  "offset": 0
}
res = requests.post(f'https://api.avito.ru/calltracking/v1/getCalls/', headers=headers, json=body)
res.json()


In [ ]:
# Получаем id звонков
call_id_users = [a['callId'] for a in res.json()['calls']]
call_id_users